In [3]:
#imports
from datetime import datetime
import pandas as pd
import numpy as np
import psycopg2

In [4]:
#changing globals

#the date that gets grabed from the excel workbook
date_to_grab = "2023-01-20"

#Midwest USA steel plate from CRU
steelPlate = 1438

#Chian Steel Plate from CRU
chinaPlate = 4350

In [5]:
#globals

# filepath = "C:\\Example\\FileName.xlsx"

#Temp file path for manual entry
filepath = "C:\\Example\\FileName.xlsx"


#column headers inside the database, this is used to create the manualCheckDF
columns = ['dates','global','china_to_us_west_coast','china_to_us_east_coast','europe_to_us_east_coast',\
            'lme_copper_spot_','lme_aluminum_spot_', 'generic_1st_hrc_future', 'hot_rolled_coil_shanghai',\
            'an_steel_hrc_exwrks','us_midwest_steel_rebar_exwork','lme_steel_rebar','generic_1st_jbp_future',\
            'import_fine_ore', 'lme_nickel_spot_','cn_li2co3_99_del','cn_li2co3_995_del',\
            'lithium_carbonate_china_bat','na_lithium_carbonate_cif_swap','cn_lithium_iron_phosphate_li_3',\
            'solar_grade_polysilicon_price','generic_12th_lp_future','generic_24th_lp_future','generic_36th_lp_future',\
            'generic_12th_la_future','generic_24th_la_future','generic_36th_la_future','generic_6th_hrc_future',\
            'generic_12nd_hrcfuture','generic_15th_hrcfuture','generic_3rd_lfa_future','generic_6th_lfa_future',\
            'generic_12th_lfa_futur','ep_resins_us_liquid_contr_ddp','ep_resins_us_solid_contr_ddp','china_renminbi',\
            'johnson_matthey_platinum_price','johnson_matthey_iridium_price','lme_cobalt_spot_','lme_cobalt_3mo_',\
            'lme_cobalt_15mo_','shanghai_manganese_spot','cn_lioh_565_del','lme_zinc_spot_','shanghai_zinc_0_spot',\
            'steel_plate_usa_midwest_cru','steel_plate_china', 'pv_cell_weekly_spot_polysilicon_price']


#a tempalte that gets formated with the numbers pulled form the execl book, its then run and thrown into the database with this query
queryTemplate = """INSERT INTO commoditydata(Dates,Global,China_to_US_West_Coast,China_to_US_East_Coast,
Europe_to_US_East_Coast,LME_COPPER_SPOT_,LME_ALUMINUM_SPOT_,Generic_1st_HRC_Future,Hot_Rolled_Coil_Shanghai,
AN_Steel_HRC_ExWrks,US_Midwest_Steel_Rebar_ExWork,LME_STEEL_REBAR,Generic_1st_JBP_Future,Import_Fine_Ore,
LME_NICKEL_SPOT_,CN_Li2CO3_99_DEL,CN_Li2CO3_995_DEL,Lithium_Carbonate_China_Bat,NA_Lithium_Carbonate_CIF_Swap,
CN_Lithium_Iron_Phosphate_Li_3,Solar_Grade_Polysilicon_Price,Generic_12th_LP_Future,Generic_24th_LP_Future,Generic_36th_LP_Future,
Generic_12th_LA_Future,Generic_24th_LA_Future,Generic_36th_LA_Future,Generic_6th_HRC_Future,Generic_12nd_HRCFuture,
Generic_15th_HRCFuture,Generic_3rd_LFA_Future,Generic_6th_LFA_Future,Generic_12th_LFA_Futur,Ep_Resins_US_Liquid_Contr_DDP,
Ep_Resins_US_Solid_Contr_DDP,CHINA_RENMINBI,Johnson_Matthey_Platinum_Price,Johnson_Matthey_Iridium_Price,
LME_COBALT_SPOT_,LME_COBALT_3MO_,LME_COBALT_15MO_,Shanghai_Manganese_Spot,CN_LiOH_565_DEL,LME_ZINC_SPOT_,
Shanghai_Zinc_0_Spot,Steel_Plate_USA_Midwest_CRU,Steel_Plate_China, pv_cell_weekly_spot_polysilicon_price) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s);"""

metricTonToUSTon = 1.10231
tariff = 1.25

In [6]:
#connect to the database
conn = psycopg2.connect(
    host = "aws-example.amazonaws.com",
    database = "DBname",
    port = "2222",
    user = "example_UN",
    password = "798645312")

In [7]:
#read the excel file at the specified path, Grabs the worksheet in the second position
DF = pd.read_excel(filepath, 1)

In [8]:
#grab row within the read excel sheet where the third column matches the date_to_grab value above
row_to_grab = DF[DF["Unnamed: 3"]==datetime.fromisoformat(date_to_grab)].index[0]

In [9]:
#grabs that row and turns it into a numpy list
inputArr = DF.iloc[row_to_grab].to_numpy()
inputArr = inputArr[3:49].tolist()

In [10]:
#The current RMB per USD is grabed from the list and then china plate is converted to USD with an added on tariff of 1.25
conversionRateRMB = inputArr[35]
chinaPlateConverted = round(((chinaPlate/conversionRateRMB)/metricTonToUSTon) * 1.25, 2)

#Polysilicon Cell is grabed from the 45 index and stored for later
PolyCellPrice = inputArr[45]

In [11]:
#add steelPlate to the 45 index overidding polysilicon then adding both china plate nad polysilicon back onto the list
# this is done to make the list match the postions in the database
inputArr[45] = steelPlate
inputArr.append(chinaPlateConverted)
inputArr.append(PolyCellPrice)

In [12]:
#creates a DF and adds the databases columns to be able to easily manualy check the input arr works
manualCheckDF = pd.DataFrame(inputArr).T
manualCheckDF.columns = columns
manualCheckDF

,dates,global,china_to_us_west_coast,china_to_us_east_coast,europe_to_us_east_coast,lme_copper_spot_,lme_aluminum_spot_,generic_1st_hrc_future,hot_rolled_coil_shanghai,an_steel_hrc_exwrks,...,lme_cobalt_spot_,lme_cobalt_3mo_,lme_cobalt_15mo_,shanghai_manganese_spot,cn_lioh_565_del,lme_zinc_spot_,shanghai_zinc_0_spot,steel_plate_usa_midwest_cru,steel_plate_china,pv_cell_weekly_spot_polysilicon_price
0,2023-01-20,2238,1361,2812,5507,9301,2558.25,718,4210,762.5,...,48574,49000,50800,17000,492500,3474,24470,1438,726.94,150


In [13]:
#cursor class to enable queries
cur = conn.cursor()

In [14]:
#The query template string is and insert query formated with %s 
#by adding a list to the execute function the query template is automaticaly formated subing out all the %s with values from the list based on position inside the list
#for more info check here https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-execute.html
cur.execute(queryTemplate,inputArr)

UniqueViolation: duplicate key value violates unique constraint "commoditydata_pkey"
DETAIL:  Key (dates)=(2023-01-20) already exists.


In [15]:
# grabs the commodity data 
cur.execute("SELECT * FROM commoditydata order by dates DESC limit(10);")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [16]:
#spits out todays update to double check it worked
cur.fetchone()

ProgrammingError: no results to fetch

In [17]:
#commit the changes then close the cursor and the connection
conn.commit()
cur.close()
conn.close()

In [18]:
conn.close()